In [1]:
import glob
import os
import librosa
import matplotlib.pyplot as plt
from datetime import datetime
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
%matplotlib inline
plt.style.use('ggplot')

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls "drive/My Drive/Dataset"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Test  test.csv	Train  train.csv


In [0]:
train  = pd.read_csv('drive/My Drive/Dataset/train.csv')
data_dir = 'drive/My Drive/Dataset'


In [0]:
def parser(row):
   # function to load files and extract features
  file_name = os.path.join(os.path.abspath(data_dir), 'Train', str(row.ID) + '.wav')

   # handle exception to check if there isn't a file which is corrupted
  try:
      # here kaiser_fast is a technique used for faster extraction
      X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
      # we extract mfcc feature from data
      mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
  except Exception as e:
      print("Error encountered while parsing file: ", file)
      return None, None
 
  feature = mfccs
  label = row.Class
 
  return [feature, label]

temp = train.apply(parser, axis=1)
temp.columns = ['feature', 'label']

In [6]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
train_x = np.array(temp.feature.tolist())
train_y = np.array(temp.label.tolist())

lb = LabelEncoder()

train_y = tf.keras.utils.to_categorical(lb.fit_transform(train_y))
print(type(train_y))

<class 'numpy.ndarray'>


In [8]:
from numpy import newaxis
train_x.shape
train_x = train_x[:,:,newaxis]
print(train_x.shape)

(5435, 40, 1)


In [9]:
train_y

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [0]:
class CausalConv1D(tf.layers.Conv1D):
    def __init__(self, filters,
               kernel_size,
               strides=1,
               dilation_rate=1,
               activation=None,
               use_bias=True,
               kernel_initializer=None,
               bias_initializer=tf.zeros_initializer(),
               kernel_regularizer=None,
               bias_regularizer=None,
               activity_regularizer=None,
               kernel_constraint=None,
               bias_constraint=None,
               trainable=True,
               name=None,
               **kwargs):
        super(CausalConv1D, self).__init__(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding='valid',
            data_format='channels_last',
            dilation_rate=dilation_rate,
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            trainable=trainable,
            name=name, **kwargs
        )
        
    def call(self, inputs):
        padding = (self.kernel_size[0] - 1) * self.dilation_rate[0]
        if self.data_format == 'channels_first':
            inputs = tf.pad(inputs, tf.constant([[0, 0], [0, 0], [padding, 0]], dtype=tf.int32))
        else:
            inputs = tf.pad(inputs, tf.constant([(0, 0,), (padding, 0), (0, 0)]))
        return super(CausalConv1D, self).call(inputs), inputs

In [11]:
tf.reset_default_graph()
with tf.Graph().as_default() as g:
    x = tf.random_normal((32, 10, 4)) # (batch_size, length, channel)
    with tf.variable_scope("tcn"):
        conv = CausalConv1D(8, 3, activation=tf.nn.relu)
    output = conv(x)
    init = tf.global_variables_initializer()
    
with tf.Session(graph=g) as sess:
    # Run the initializer
    sess.run(init)
    res, inputs = sess.run(output)
    print(inputs.shape)
    print(inputs[0, :, 0])
    print(res.shape)    
    print(res[0, :, 0])

(32, 12, 4)
[ 0.          0.          0.19434792  0.62958175  1.5533562   0.9575418
 -0.08206888  0.03701568  0.7041562   1.2063874  -0.65986335  0.10659102]
(32, 10, 8)
[0.        0.        0.        0.        0.0213389 0.        0.
 0.        0.        0.       ]


In [12]:
tf.reset_default_graph()
with tf.Graph().as_default() as g:
    x = tf.expand_dims(
        tf.expand_dims(tf.constant([1, 0, 0, 1, 0, 0, 1], dtype=tf.float32), axis=0),
        axis=-1) # (batch_size, length, channel)
    with tf.variable_scope("tcn"):
        conv = CausalConv1D(8, 2, dilation_rate=2, activation=None)
    output = conv(x)
    init = tf.global_variables_initializer()
    
with tf.Session(graph=g) as sess:
    # Run the initializer
    sess.run(init)
    res, inputs = sess.run(output)
    print(inputs.shape)
    print(inputs[0, :, 0])
    print(res.shape)    
    print(res[0, :, 0])

(1, 9, 1)
[0. 0. 1. 0. 0. 1. 0. 0. 1.]
(1, 7, 8)
[-0.2616848  0.         0.3173225 -0.2616848  0.         0.3173225
 -0.2616848]


In [13]:
tf.reset_default_graph()
with tf.Graph().as_default() as g:
    x = tf.random_normal((32, 4, 10)) # (batch_size, channel, length)
    dropout = tf.layers.Dropout(0.5, noise_shape=[x.shape[0], x.shape[1], tf.constant(1)])
    output = dropout(x, training=True)
    init = tf.global_variables_initializer()
    
with tf.Session(graph=g) as sess:
    # Run the initializer
    sess.run(init)
    res = sess.run(output)
    print(res.shape)   
    print(res[0, :, :])
    print(res[1, :, :])

(32, 4, 10)
[[-2.3285773   0.08641841 -1.4585888  -0.653349   -0.06835803 -0.9478375
   1.4381355  -3.091065   -1.319882   -1.5865655 ]
 [-0.         -0.          0.         -0.          0.         -0.
  -0.          0.         -0.          0.        ]
 [-0.          0.          0.         -0.          0.         -0.
   0.         -0.          0.         -0.        ]
 [-1.0714962   1.1951921   0.99483836  1.7352117  -2.016487   -1.8198144
   2.097631    0.7876429   0.05295105  2.3481727 ]]
[[-0.40814415 -1.2866764  -0.5079775  -0.977586   -0.08755136 -1.3722992
  -3.667378    0.14989467  2.2631004   1.4908382 ]
 [-3.100421    1.6499041   1.2775149  -0.43227136  0.7758265   1.6978185
   0.9040202  -1.8233161  -2.1887565   1.0640903 ]
 [-0.         -0.         -0.         -0.          0.         -0.
  -0.         -0.          0.          0.        ]
 [ 0.7119539   0.68701047 -0.9982313  -0.86383283  0.87027013  0.87300307
  -2.3336658  -1.554365    1.7374347  -2.6428702 ]]


In [0]:
# Redefining CausalConv1D to simplify its return values
class CausalConv1D(tf.layers.Conv1D):
    def __init__(self, filters,
               kernel_size,
               strides=1,
               dilation_rate=1,
               activation=None,
               use_bias=True,
               kernel_initializer=None,
               bias_initializer=tf.zeros_initializer(),
               kernel_regularizer=None,
               bias_regularizer=None,
               activity_regularizer=None,
               kernel_constraint=None,
               bias_constraint=None,
               trainable=True,
               name=None,
               **kwargs):
        super(CausalConv1D, self).__init__(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding='valid',
            data_format='channels_last',
            dilation_rate=dilation_rate,
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            trainable=trainable,
            name=name, **kwargs
        )
       
    def call(self, inputs):
        padding = (self.kernel_size[0] - 1) * self.dilation_rate[0]
        inputs = tf.pad(inputs, tf.constant([(0, 0,), (1, 0), (0, 0)]) * padding)
        return super(CausalConv1D, self).call(inputs)

In [0]:
class TemporalBlock(tf.layers.Layer):
    def __init__(self, n_outputs, kernel_size, strides, dilation_rate, dropout=0.2, 
                 trainable=True, name=None, dtype=None, 
                 activity_regularizer=None, **kwargs):
        super(TemporalBlock, self).__init__(
            trainable=trainable, dtype=dtype,
            activity_regularizer=activity_regularizer,
            name=name, **kwargs
        )        
        self.dropout = dropout
        self.n_outputs = n_outputs
        self.conv1 = CausalConv1D(
            n_outputs, kernel_size, strides=strides, 
            dilation_rate=dilation_rate, activation=tf.nn.relu, 
            name="conv1")
        self.conv2 = CausalConv1D(
            n_outputs, kernel_size, strides=strides, 
            dilation_rate=dilation_rate, activation=tf.nn.relu, 
            name="conv2")
        self.down_sample = None

    
    def build(self, input_shape):
        channel_dim = 2
        self.dropout1 = tf.layers.Dropout(self.dropout, [tf.constant(1), tf.constant(1), tf.constant(self.n_outputs)])
        self.dropout2 = tf.layers.Dropout(self.dropout, [tf.constant(1), tf.constant(1), tf.constant(self.n_outputs)])
        if input_shape[channel_dim] != self.n_outputs:
            # self.down_sample = tf.layers.Conv1D(
            #     self.n_outputs, kernel_size=1, 
            #     activation=None, data_format="channels_last", padding="valid")
            self.down_sample = tf.layers.Dense(self.n_outputs, activation=None)
        self.built = True
    
    def call(self, inputs, training=True):
        x = self.conv1(inputs)
        x = tf.contrib.layers.layer_norm(x)
        x = self.dropout1(x, training=training)
        x = self.conv2(x)
        x = tf.contrib.layers.layer_norm(x)
        x = self.dropout2(x, training=training)
        if self.down_sample is not None:
            inputs = self.down_sample(inputs)
        return tf.nn.relu(x + inputs)

In [16]:
tf.reset_default_graph()
with tf.Graph().as_default() as g:
    x = tf.random_normal((32, 10, 4)) # (batch_size, length, channel)
    tblock = TemporalBlock(8, 2, 1, 1)
    output = tblock(x, training=tf.constant(True))
    init = tf.global_variables_initializer()
    
with tf.Session(graph=g) as sess:
    # Run the initializer
    sess.run(init)
    res = sess.run(output)
    print(res.shape)   
    print(res[0, :, 0])
    print(res[1, :, 1])

(32, 10, 8)
[0.        0.        0.        0.2258923 0.        0.        0.
 0.        0.3263411 3.5823278]
[0.         0.         0.         0.         4.0272355  0.
 0.         0.         1.933734   0.35593283]


In [0]:
class TemporalConvNet(tf.layers.Layer):
    def __init__(self, num_channels, kernel_size=2, dropout=0.2,
                 trainable=True, name=None, dtype=None, 
                 activity_regularizer=None, **kwargs):
        super(TemporalConvNet, self).__init__(
            trainable=trainable, dtype=dtype,
            activity_regularizer=activity_regularizer,
            name=name, **kwargs
        )
        self.layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            out_channels = num_channels[i]
            self.layers.append(
                TemporalBlock(out_channels, kernel_size, strides=1, dilation_rate=dilation_size,
                              dropout=dropout, name="tblock_{}".format(i))
            )
    
    def call(self, inputs, training=True):
        outputs = inputs
        for layer in self.layers:
            outputs = layer(outputs, training=training)
        return outputs

In [18]:
tf.reset_default_graph()
with tf.Graph().as_default() as g:
    x = tf.random_normal((32, 10, 4)) # (batch_size, length, channel)
    tcn = TemporalConvNet([8, 8, 8, 8], 2, 0.25)
    output = tcn(x, training=tf.constant(True))
    init = tf.global_variables_initializer()
    
with tf.Session(graph=g) as sess:
    # Run the initializer
    sess.run(init)
    res = sess.run(output)
    print(res.shape)   
    print(res[0, :, 0])
    print(res[1, :, 1])

(32, 10, 8)
[ 0.1667993   0.754282    0.          0.32326317  0.66547775  1.4692276
  0.          2.2628198   0.6556042  12.017203  ]
[0.        0.        0.        1.5356847 0.        0.        0.
 0.        0.        0.       ]


In [19]:
tf.reset_default_graph()
g = tf.Graph()
with g.as_default():
    Xinput = tf.placeholder(tf.float32, shape=[None, 10, 4])
    tcn = TemporalConvNet([8, 8, 8, 8], 2, 0.25)
    output = tcn(Xinput, training=tf.constant(True))
    print(tcn.layers[0].down_sample)    
    init = tf.global_variables_initializer()
    
with tf.Session(graph=g) as sess:
    # Run the initializer
    sess.run(init)
    res = sess.run(output, {Xinput: np.random.randn(32, 10, 4)})
    print(res.shape)   
    print(res[0, :, 0])
    print(res[1, :, 1])

(32, 10, 8)
[0.        0.        0.        8.308353  0.        1.1927321 0.
 7.328927  6.311587  0.       ]
[0.         1.2304927  0.         1.2514353  0.         8.906823
 0.         0.9208402  0.28090048 1.5971172 ]


In [20]:
# Training Parameters
learning_rate = 0.001
batch_size = 32
display_step = 500
total_batch = int(5435 / batch_size)
print("Number of batches per epoch:", total_batch)
training_steps = 2000

# Network Parameters
num_input = 40 
timesteps = 1 # timesteps
num_classes = 10 #  total classes (0-9 digits)
dropout = 0.1
kernel_size = 8
levels = 6
nhid = 20 # hidden layer num of features

Number of batches per epoch: 169


In [21]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(10)
    # tf Graph input
    X = tf.placeholder("float", [None, num_input, timesteps])
    Y = tf.placeholder("float", [None, num_classes])
    is_training = tf.placeholder("bool")
    
    # Define weights
    logits = tf.layers.dense(
        TemporalConvNet([nhid] * levels, kernel_size, dropout)(
            X, training=is_training)[:, -1, :],
        num_classes, activation=None, 
        kernel_initializer=tf.orthogonal_initializer()
    )
    prediction = tf.nn.softmax(logits)
   
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, labels=Y))
    
    with tf.name_scope("optimizer"):
        # optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        # gvs = optimizer.compute_gradients(loss_op)
        # for grad, var in gvs:
        #     if grad is None:
        #         print(var)
        # capped_gvs = [(tf.clip_by_value(grad, -.5, .5), var) for grad, var in gvs]
        # train_op = optimizer.apply_gradients(capped_gvs)    
        train_op = optimizer.minimize(loss_op)

    # Evaluate model (with test logits, for dropout to be disabled)
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    print("All parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.global_variables()]))
    print("Trainable parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.trainable_variables()]))

All parameters: 108992.0
Trainable parameters: 36330


In [27]:


config = tf.ConfigProto()
config.gpu_options.allow_growth = False
best_val_acc = 0.8
with tf.Session(graph=graph, config=config) as sess:
    # Run the initializer
    sess.run(init)
    for step in range(1, training_steps+1):
        
        offset = (step * batch_size) % (train_y.shape[0] - batch_size)
        batch_x = train_x[offset:(offset + batch_size)]
        batch_y = train_y[offset:(offset + batch_size), :]
        
        
        
         
           
          
         
       
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, is_training: True})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={
                X: batch_x, Y: batch_y, is_training: False})
            
            print ("Iter " + str(step) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))


    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in path: %s" % save_path)
    print("Optimization Finished!")

Iter 1, Minibatch Loss= 2.280263, Training Accuracy= 0.15625
Iter 500, Minibatch Loss= 1.545001, Training Accuracy= 0.53125
Iter 1000, Minibatch Loss= 0.879534, Training Accuracy= 0.75000
Iter 1500, Minibatch Loss= 0.923773, Training Accuracy= 0.68750
Iter 2000, Minibatch Loss= 0.234974, Training Accuracy= 0.96875
Model saved in path: /tmp/model.ckpt
Optimization Finished!
